# Welcome to the pywave library

Assuming translational symmetry of initial condition this program solves schrodinger's equation for a 1D particle for any time indepentdent potentials. The method that I am using relies of solving for the matrix equavatlent of the double spacial derivative of a free particle then adding a diagonilized matrix formed from the chosen system's potentials and then solving for the eigenvalues and eigenvectos from this addition. Then using the ANIMATE function we can view how our chosen initial condition changes in time for the chosen system's potentials.
Also using the EIG_ANALYSIS function we have view all the eigenvalues and eigenvectors and get more insights about the different energy states of our system.  


This program solves the time-independent Schrödinger's equation for a one-dimensional particle, assuming translational symmetry of the initial condition. The method involves finding the matrix equivalent of the second spatial derivative and adding a diagonalized matrix representing the potential energy of the chosen system. The eigenvalues and eigenvectors of the resulting matrix are then calculated.

The program includes several key functions for setting up and analyzing the system:

* WAVE_1D_FUNCTION(n, f, V, mV, L=1, CORRECT_UNITS=True)

    * This is the core function that solves the Schrödinger equation.
    
    * It takes as input the number of spatial points n, the initial wavefunction f, the potential V, the mass mV, the length of the system L, and a boolean CORRECT_UNITS to toggle physical constants.
    
    * It calculates the matrix for the kinetic energy operator in Fourier space and adds the potential energy matrix.
    
    * It then solves for the eigenvalues (energy states) and eigenvectors (energy eigenfunctions) of the total Hamiltonian matrix.
    
    * Finally, it determines the coefficients to express the initial wavefunction as a linear combination of these eigenvectors.
    
    * The function returns a new function, WAVE_FUNCTION(t), which can be called with a specific time t to get the spatial distribution of the particle's probability density at that time.

* ANIMATE(SYST, V, L=500, dt=10)
    * This function allows for the visualization of how a chosen initial condition evolves over time under the influence of the system's potential. It takes the system information, the potential, the total animation time, and the time step as inputs.

* EIG_ANALYSIS(Es, Evects, As, x, size=(8, 6))
    * This function provides a way to view all the eigenvalues and eigenvectors, offering deeper insights into the different energy states of the system. It displays the eigenvectors and the magnitudes of their coefficients.








In [10]:
from scipy.constants import hbar,electron_mass 
from numpy.linalg import eig,inv,solve,eigh
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
inveig = lambda Es,Evects: Evects@(np.diag(Es))@inv(Evects)
π,ħ,me = np.pi,hbar,electron_mass
def gaussian(x, μ, σ):
    return (
        1.0 / (np.sqrt(2.0 * π) * σ) * np.exp(-np.power((x - μ) / σ, 2.0) / 2)
    )
def WAVE_1D_FUNCTION(n,f,V,mV,L=1,CORRECT_UNITS=True):
    x,Δx = np.linspace(0, L, n,endpoint=False, retstep=True)
    freqs = np.fft.fftfreq(n, Δx)
    ks = 2*π*freqs
    Evects = np.array([np.exp(1j*k*x) for k in ks]).T
    Es = ks**2*(ħ**2) / (2 * me) if CORRECT_UNITS else ks**2
    MATRIX = inveig(Es,Evects)
    eigenvalues,eigenvectors = eig(MATRIX + np.diag(V(x)*mV))
    As=solve(eigenvectors, f(x))
    def WAVE_FUNCTION(t):
        y = np.abs(eigenvectors@(As*np.exp(-1j*t*eigenvalues/ħ)))**2
        return x,y
    return WAVE_FUNCTION
def WAVE_1D_MAT(n,L=1,CORRECT_UNITS=True):
    x,Δx = np.linspace(0, L, n,endpoint=False, retstep=True)
    freqs = np.fft.fftfreq(n, Δx)
    ks = 2*π*freqs
    Evects = np.array([np.exp(1j*k*x) for k in ks]).T
    Es = ks**2*(ħ**2) / (2 * me) if CORRECT_UNITS else ks**2
    MATRIX = inveig(Es,Evects)
    return MATRIX
def WAVE_1D_EIG(n,f,V,mV,L=1,CORRECT_UNITS=True):
    x,Δx = np.linspace(0, L, n,endpoint=False, retstep=True)
    freqs = np.fft.fftfreq(n, Δx)
    ks = 2*π*freqs
    Evects = np.array([np.exp(1j*k*x) for k in ks]).T
    Es = ks**2*(ħ**2) / (2 * me) if CORRECT_UNITS else ks**2
    MATRIX = inveig(Es,Evects)
    eigenvalues,eigenvectors = eig(MATRIX + np.diag(V(x)*mV))
    As=solve(eigenvectors, f(x))
    return eigenvalues,eigenvectors,As,x

def ANIMATE(SYST,V,L = 500,dt = 10):
    %matplotlib widget
    Es,Evects,As,x = SYST
    fig, ax = plt.subplots()
    line, = ax.plot([], [])
    line2, = ax.plot([], [])
    MAX = np.max(np.abs(Evects@As)**2)
    ax.set_xlim(min(x), max(x))
    ax.set_ylim(0, MAX)
    def animateWave(frame):
        t = frame*dt
        y = np.abs(Evects@(As*np.exp(-1j*t*Es/ħ)))**2
        line.set_data(x, y)
        line2.set_data(x, V(x))
        ax.set_title(f"Time: {t} s")
        return line,line2,
    anim = FuncAnimation(fig, animateWave, frames=int(L/dt), interval=20)
    return HTML(anim.to_jshtml())
def animate_multiple(*data_funcs, frames=300, interval=20,dt=10):
    %matplotlib widget
    fig, ax = plt.subplots()
    lines = [ax.plot([], [])[0] for _ in data_funcs]
    def init():
        x0,y0 = list(data_funcs)[0](0)
        ax.set_xlim(min(x0), max(x0))
        ax.set_ylim(min(y0), max(y0))
        for line in lines:
            line.set_data([], [])
        return lines
    def update(frame):
        for line, func in zip(lines, data_funcs):
            x, y = func(frame*dt)
            line.set_data(x, y)
        ax.set_title(f"Time: {frame*dt} s")
        return lines
    ani = FuncAnimation(
        fig, update, init_func=init, frames=frames,
        interval=interval, blit=True
    )
    return HTML(ani.to_jshtml())
def EIG_ANALYSIS(Es,Evects,As,x,size = (8,6)):
    %matplotlib inline
    order = np.argsort(Es)
    Es,Evects,As = Es[order],Evects.T[order],As[order]
    # Create figure and axes layout
    fig = plt.figure(figsize=size)
    gs = fig.add_gridspec(2, 3, height_ratios=[2, 1], width_ratios=[1, 1, 0.05])
    # Top plot spanning both columns
    ax1 = fig.add_subplot(gs[0, :2])
    for i in range(0,len(Es)):
        plt.plot(x,np.abs(Evects[i]*As[i]))
    ax1.set_title('EigenVectors for all EigenValues')
    # Bottom left scatter plot
    ax2 = fig.add_subplot(gs[1, 0])
    sc2_left = ax2.scatter(np.abs(Es), np.abs(As), c=np.abs(Es), cmap='viridis',s=10)
    ax2.set_yscale('log')
    ax2.set_title('Log Magnitudes by Energy')
    # Bottom right scatter plot
    ax3 = fig.add_subplot(gs[1, 1])
    sc2_right = ax3.scatter(range(len(As)),  np.abs(As), c=np.abs(Es), cmap='viridis',s=10)
    #ax3.set_yscale('log')
    ax3.set_title('Magnitudes by Indeces')
    cax = fig.add_subplot(gs[1, 2])
    plt.colorbar(sc2_right, cax=cax, label='Shared Color Scale')
    plt.tight_layout()
    plt.show()